# **UB** : Save and Restore Crystal Orientation

see: https://github.com/bluesky/hklpy/issues/50

**Objectives**

1. Save the information defining the crystal orientation into the descriptor
1. List runs that have orientation that can be restored
1. Restore crystal orientation from a given Bluesky run

------------
# Data collection

## Setup for data collection

Use a local, temporary, file-based databroker.  It will reset after each restart of the notebook.  Prepare to define the diffractometers needed here plus some items from the ophyd simulators.

In [1]:
import gi
gi.require_version('Hkl', '5.0')

from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
import bluesky.plans as bp
import bluesky.plan_stubs as bps
import bluesky.preprocessors as bpp
import databroker
import hkl
from hkl.calc import A_KEV
from hkl.util import Lattice, run_orientation_info, list_orientation_runs
from hkl.geometries import *
import numpy as np
import pyRestTable
from ophyd import Component, Device, EpicsSignal, Signal
from ophyd.signal import AttributeSignal, ArrayAttributeSignal
from ophyd.sim import *
import pandas as pd

bec = BestEffortCallback()
bec.disable_plots()
cat = databroker.temp().v2

RE = RunEngine({})
RE.subscribe(bec)
RE.subscribe(cat.v1.insert)
RE.md["notebook"] = "UB_save_restore"
RE.md["objective"] = "Demonstrate UB matrix save & restore"

/home/mintadmin/Apps/anaconda/envs/bluesky_2021_1/lib/python3.8/site-packages/databroker/core.py:1856: UserWarning: There are 2 entrypoints for the databroker handler spec 'NPY_SEQ'. They are [EntryPoint('NPY_SEQ', 'ophyd.sim', 'NumpySeqHandler', Distribution('ophyd', '1.6.0')), EntryPoint('NPY_SEQ', 'ophyd.sim', 'NumpySeqHandler', None)]. The match EntryPoint('NPY_SEQ', 'ophyd.sim', 'NumpySeqHandler', Distribution('ophyd', '1.6.0')) has won the race.
  warnings.warn(


-------------

## Build simulated 4-circle diffractometer

Build two 4-circles so that we can test routines that differentiate between similar diffractometers.  Use the second one to restore orientation saved from the first.

In [2]:
class Fourc(SimulatedE4CV):
    pass

fourc = Fourc("", name="fourc")
fourc.energy.put(A_KEV / 1.54)
a0 = 5.4310196
fourc.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
fourc.calc.sample.compute_UB(
    fourc.calc.sample.add_reflection(4, 0, 0, (-145.451, 0, 0, 69.0966)),
    fourc.calc.sample.add_reflection(0, 4, 0, (-145.451, 0, 90, 69.0966))
)
fourc.pa()

orange = Fourc("", name="orange")
orange.pa()

===================== ===========================================================================
term                  value                                                                      
===================== ===========================================================================
diffractometer        fourc                                                                      
geometry              E4CV                                                                       
class                 Fourc                                                                      
energy (keV)          8.05092                                                                    
wavelength (angstrom) 1.54000                                                                    
calc engine           hkl                                                                        
mode                  bissector                                                                  
positions           

Build simulators for other diffractometer geometries to test code that differentiates between various possibile sources for restore of orientation information.

In [4]:
class Kappa(SimulatedK4CV):
    pass

kappa = Kappa("", name="kappa")
kappa.energy.put(A_KEV / 1.54)
a0 = 5.4310196
kappa.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
kappa.calc.sample.compute_UB(
    kappa.calc.sample.add_reflection(4, 0, 0, (55.4507, 0, 90, -69.0966)), 
    kappa.calc.sample.add_reflection(0, 4, 0, (-1.5950, 134.7568, 123.3554, -69.0966))
)
kappa.pa()

===================== =================================================================================
term                  value                                                                            
===================== =================================================================================
diffractometer        kappa                                                                            
geometry              K4CV                                                                             
class                 Kappa                                                                            
energy (keV)          8.05092                                                                          
wavelength (angstrom) 1.54000                                                                          
calc engine           hkl                                                                              
mode                  bissector                                 

In [5]:
class Sixc(SimulatedE6C):
    pass

sixc = Sixc("", name="sixc")
sixc.energy.put(A_KEV / 1.54)
a0 = 5.4310196
sixc.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
sixc.calc.sample.compute_UB(
    sixc.calc.sample.add_reflection(4, 0, 0, (0, -145.451, 0, 0, 0, 69.0966)),
    sixc.calc.sample.add_reflection(0, 4, 0, (0, -145.451, 90, 0, 0, 69.0966))
)
sixc.pa()

===================== ========================================================================================================
term                  value                                                                                                   
===================== ========================================================================================================
diffractometer        sixc                                                                                                    
geometry              E6C                                                                                                     
class                 Sixc                                                                                                    
energy (keV)          8.05092                                                                                                 
wavelength (angstrom) 1.54000                                                                                  

## Collect data with all the diffractometers

Show data collection with and without the orientation information.

**Tip**: To save orientation information, add the diffractometer as an additional detector.  That's all!  Works with any scan that supports multiple detectors.

In [6]:
def scan_all():
    ### count runs ###
    # this run will not save orientation information
    yield from bp.count([noisy_det])
    # this run _will_ save orientation information for fourc
    yield from bp.count([noisy_det, fourc])
    # this run _will_ save orientation information for several diffractometers
    yield from bp.count([noisy_det, fourc, orange, kappa, sixc])

    ### scan runs ###
    yield from bp.scan([noisy_det], fourc.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det, fourc], fourc.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det], kappa.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det, kappa], kappa.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det], sixc.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det, sixc], sixc.h, 0.9, 1.1, 2)

    ### mesh runs at the (100) ###
    # first, move to the (100)
    yield from bps.mv(fourc.h, 1, fourc.k, 0, fourc.l, 0)
    yield from bp.rel_grid_scan([noisy_det], fourc.h, -0.1, 0.1, 3, fourc.k, -0.1, 0.1, 3)
    yield from bp.rel_grid_scan([noisy_det, fourc], fourc.h, -0.1, 0.1, 3, fourc.k, -0.1, 0.1, 3)

Run the scans, gather all the uids into a variable to be ignored.  That way, they do not print.

In [7]:
_uids = RE(scan_all())



Transient Scan ID: 1     Time: 2021-04-25 14:18:08
Persistent Unique Scan ID: 'f239605d-7404-45c2-82db-1f75717c6b8c'
New stream: 'primary'
+-----------+------------+------------+
|   seq_num |       time |  noisy_det |
+-----------+------------+------------+
|         1 | 14:18:08.3 |      1.023 |
+-----------+------------+------------+
generator count ['f239605d'] (scan num: 1)





Transient Scan ID: 2     Time: 2021-04-25 14:18:08
Persistent Unique Scan ID: '52809cf9-d6a4-4ab7-865a-47fe454bfac5'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+
|   seq_num |       time |  noisy_det |    fourc_h |    fourc_k |    fourc_l |
+-----------+------------+------------+------------+------------+------------+
|         1 | 14:18:08.4 |      1.074 |      0.000 |      0.000 |      0.000 |
+-----------+------------+------------+------------+------------+------------+
generator count ['52809cf9'] (scan num: 2)





Transient Scan ID: 3     Time

------------
# Show the orientation information that was collected

Show the full contents of the descriptor document (primary stream) for the `fourc` "detector" from the run with `scan_id=5`.  This is where the orientation information is saved.  You may need to expand the *Data variables* row to see all the orientation information.

In [8]:
cat[5].primary.config["fourc"].read()

<xarray.Dataset>
Dimensions:                    (dim_0: 6, dim_1: 6, dim_10: 6, dim_11: 21, dim_2: 3, dim_3: 3, dim_4: 3, dim_5: 3, dim_6: 2, dim_7: 3, dim_8: 4, dim_9: 4, time: 2)
Coordinates:
  * time                       (time) float64 1.619e+09 1.619e+09
Dimensions without coordinates: dim_0, dim_1, dim_10, dim_11, dim_2, dim_3, dim_4, dim_5, dim_6, dim_7, dim_8, dim_9
Data variables: (12/21)
    fourc_energy               (time) float64 8.051 8.051
    fourc_energy_units         (time) <U3 'keV' 'keV'
    fourc_energy_offset        (time) int64 0 0
    fourc_geometry_name        (time) <U4 'E4CV' 'E4CV'
    fourc_class_name           (time) <U5 'Fourc' 'Fourc'
    fourc_sample_name          (time) <U7 'silicon' 'silicon'
    ...                         ...
    fourc__hklpy_version       (time) <U25 '0.3.15+170.g3885fc9.dirty' '0.3.1...
    fourc__pseudos             (time, dim_7) <U1 'h' 'k' 'l' 'h' 'k' 'l'
    fourc__reals               (time, dim_8) <U5 'omega' 'chi' ... 'phi' 'tth'
    fourc__constraints         (time, dim_9, dim_10) <U23 'omega' ... 'False'
    fourc__mode                (time) <U9 'bissector' 'bissector'
    fourc_orientation_attrs    (time, dim_11) <U19 'orientation_attrs' ... '_...

## Show orientation that was saved

In `scan_id=3`, orientation information from 4 different diffractometers was saved with the run.  Show what is available from each of those diffractometers.  The columns in the next table are the diffractometers, the rows are the orientation information saved for each.

In [9]:
roi = run_orientation_info(cat[3])
pd.DataFrame(roi)

,fourc,kappa,sixc,orange
energy,8.05092,8.05092,8.05092,8
energy_units,keV,keV,keV,keV
energy_offset,0,0,0,0
geometry_name,E4CV,K4CV,E6C,E4CV
class_name,Fourc,Kappa,Sixc,Fourc
sample_name,silicon,silicon,silicon,main
lattice,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...","[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...","[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...","[1.54, 1.54, 1.54, 90.0, 90.0, 90.0]"
lattice_reciprocal,"[1.1569071316147683, 1.1569071316147683, 1.156...","[1.1569071316147683, 1.1569071316147683, 1.156...","[1.1569071316147683, 1.1569071316147683, 1.156...","[4.079990459207523, 4.079990459207523, 4.07999..."
U,"[[-1.2217304763832569e-05, -0.9999999999253688...","[[-1.7453292519418075e-05, -6.226958714415446e...","[[-1.2217304763832569e-05, -1.2217304762008981...","[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, ..."
UB,"[[-1.4134287010388982e-05, -1.156907131528427,...","[[-2.0191838585873458e-05, -7.2040129449779575...","[[-1.4134287010388982e-05, -1.4134287008279258...","[[4.079990459207523, -2.4982736282101165e-16, ..."


## Show runs with orientation information

Since a given run (``scan_id``) may have more than one set of orientation information, corresponding to more than one diffractometer, report for each when found.  Here, extra columns are reported for energy & units, and the crystal lattice parameters.  The names are taken from the above table.  (They must be one of the names in the `orientation_attrs` list.)

Use this type of listing to determine which **scan_id** and **diffractometer_name** has the orientation you wish to recover.  If the ``scan_id`` is not unique, identify the run with the **uid** (as a string, such as `cat["007abcd"]`).

In [10]:
list_orientation_runs(cat, "energy", "energy_units", "lattice")

,scan_id,sample_name,diffractometer_name,geometry_name,energy,energy_units,lattice,uid
0,2,silicon,fourc,E4CV,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",52809cf
1,3,silicon,fourc,E4CV,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",7e91832
2,3,silicon,kappa,K4CV,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",7e91832
3,3,silicon,sixc,E6C,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",7e91832
4,3,main,orange,E4CV,8.000000,keV,"[1.54, 1.54, 1.54, 90.0, 90.0, 90.0]",7e91832
5,5,silicon,fourc,E4CV,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",5eae0b6
6,7,silicon,kappa,K4CV,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",b949772
7,9,silicon,sixc,E6C,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",20468bf
8,10,silicon,fourc,E4CV,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",04256e6
9,11,silicon,fourc,E4CV,8.050922,keV,"[5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, ...",fb41cc9


------------
# Restore orientation information

This demo will restore the orientation information from a `fourc` run (choosing `scan_id=2`) to the `orange` diffractometer.  They have the same **geometry_name** so the information is compatible.

Then will try to share from `fourc`, `scan_id=2` with the `kappa` diffractometer.  That should not be possible since the geometries are not identical.  However, it should be possible to restore the sample and lattice.

Then, try to restore the sample and lattice when they already exist.  Check for differences and offer to update.